# ДЗ-1 dvp-4

In [8]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Генерируем датасет: 
Целевая метка - затраты на электроэнергию.
В признаки добавляем:
- Количество жильцов (residents)- генерируем рендомно в интервале 1-10
- Количество ламп в доме (lamps) - генерируем рендомно - 10-200
- Количество квадратных метров в доме (square) - генерируем рендомно - 15-100
- Целевая метка - затраты на электроэенергию: произведение кол-ва жильцов на кол-во метров квадратных, умноженное на показатель 250 (затраты на 1-го жильца на 1 кв.м. в мес), плюс расходы в мес неа электроэнергию в помещениях общественного пользования - 1000. 
Считаем, что количество ламп не влияет, влияет только количество освещаемых метров. 

In [9]:
n_samples = 500

residents = np.random.choice(9, n_samples) + 1
lamps = np.random.choice(190, n_samples) + 10
square = np.random.choice(85, n_samples) + 15

cost = residents * square * 250 + 1000

data = pd.DataFrame({'residents': residents, 'lamps': lamps, 'square': square,'cost': cost})
data.head(5)

,residents,lamps,square,cost
0,5,15,97,122250
1,6,189,24,37000
2,6,92,75,113500
3,1,104,56,15000
4,1,21,77,20250


Смотрим весовые коэффициенты линейной модели

In [11]:
from sklearn.metrics import mean_absolute_error

X = data[['residents', 'lamps','square']]
y = data['cost']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['residents', 'lamps','square']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [1.37878692e+04 2.68571649e+00 1.22995865e+03]
Bias: -65885.51234721893
Error: 11316.451283687305


Видим, что у lamps весосвой коэффициент много ниже весов residents и square.
Исключим lamps из анализа 

In [13]:
from sklearn.metrics import mean_absolute_error

X = data[['residents', 'square']]
y = data['cost']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['residents', 'square']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [13789.37356165  1229.80806998]
Bias: -65603.55194923075
Error: 11315.87399884007


Видим, что веса residents и square примерно отличаются в 10 раз. Как и данные их series. Предположим, что необоходимо ввести новый признак, являющийся их произведением.

In [15]:
data['mult'] = data['residents'] * data['square']

X = data[['mult']]
y = data['cost']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['mult']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [250.]
Bias: 1000.0000000000146
Error: 7.201379048638046e-12


Предположение подтвердилось - введение вновь созданного признака было удачным решением. Ошибка результата анализа стремится к нулю.
При этом мы понимаем, что у линейной функции множителем рабочего признака будет 250, а смещение по оси y - 1000.